TO DO

- is it possible to add captaincy bonus?
- add an optional constraint to allow only one defender from any given team (risk mitigation)?

In [1]:
import pandas as pd
import numpy as np
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [2]:
# function to solve the optimization problem
def optimize(df, max_price, expected_column):
    
    # PRELIMINARIES
    
    # Create the model
    model = LpProblem(name="FPL", sense=LpMaximize)    
    variables = [LpVariable(name=f'{ix}', cat='Binary') for ix in df.index]
    prices = [df.loc[ix,'now_cost']/10.0 for ix in df.index]
    # measure of player quality
    expected_points = [df.loc[ix,expected_column] for ix in df.index]
    goalkeepers = [1.0 if df.loc[ix,'element_type']==1 else 0.0 for ix in df.index]
    defenders = [1.0 if df.loc[ix,'element_type']==2 else 0.0 for ix in df.index]
    midfielders = [1.0 if df.loc[ix,'element_type']==3 else 0.0 for ix in df.index]
    forwards = [1.0 if df.loc[ix,'element_type']==4 else 0.0 for ix in df.index]
    teams = []
    for team in df['team'].unique():
        teams.append( [1.0 if df.loc[ix,'team']==team else 0.0 for ix in df.index] )

    # CONSTRAINTS

    # select 11 players
    model += lpSum(variables) == 11
    # set maximum price for starting 11
    model += np.dot(prices,variables) <= max_price
    # only 1 goalkeeper
    model += np.dot(goalkeepers,variables) == 1
    # at least 3 defenders
    model += np.dot(defenders,variables) >= 3
    # at most 5 defenders
    model += np.dot(defenders,variables) <= 5
    # at most 5 midfielders
    model += np.dot(midfielders,variables) <= 5
    # at least 1 forward
    model += np.dot(forwards,variables) >= 1
    # at most 3 forwards
    model += np.dot(forwards,variables) <= 3
    # max 3 players from any given team
    for team in teams:
        model += np.dot(team,variables) <= 3
    
    # OBJECTIVE
    # if possible, SHOULD ADD CAPTAIN'S DOUBLE POINTS TO THE OBJECTIVE
    model += np.dot(expected_points,variables)

    # SOLVE OPTIMIZATION
    
    status = model.solve()
    print(f'Status: {LpStatus[model.status]}')
    #print(f'Mean total points per gameweek: {model.objective.value()}')

    players = [int(str(var)) for var in model.variables() if var.value()==1]
    dream_team = df.loc[players]
    cost = (dream_team['now_cost']/10.0).sum()
    exp_points = dream_team[expected_column].sum() + dream_team[expected_column].max()
    print(f'Cost: {cost}')
    if expected_column == 'adjusted points per game':
        print(f'Expected points per week: {exp_points}')
    elif expected_column == 'xPoints_next10':
        exp_points = exp_points / np.min((10, 38-gameweek))
        print(f'Expected points per week: {exp_points}')
    else:
        print(f'Expected points: {exp_points}')
    display(dream_team[['element_type','web_name','now_cost','games played',expected_column]].sort_values('element_type'))

In [3]:
gameweek = 37
# fetch FPL data
filepath = '../data/fpl/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)
df

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week12,assists_week13,assists_week14,assists_week15,...,xPoints week 37,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,xPoints_next10,yellow_cards,gameweek 47 prediction
id,,,,,,,,,,,,,,,,,,,,,
1,10.602699,2.650675,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.936801,0,0.0
2,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.296703,0,0.0
3,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.336736,0,0.0
4,50.411000,3.600786,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,10.5,3.6,3.8,2.4,6.8,4.071,2.988724,3,0.0
5,51.013493,2.550675,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.300000,1.0,NaN,NaN,NaN,NaN,NaN,3.220320,3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.911306,0,0.0
685,8.171183,2.723728,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.690127,NaN,NaN,NaN,NaN,NaN,NaN,2.001500,1,0.0
686,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.858341,0,0.0


In [4]:
# only consider players that have played at least some minimum number of games
df = df[df['games played']>=5]
df

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week12,assists_week13,assists_week14,assists_week15,...,xPoints week 37,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,xPoints_next10,yellow_cards,gameweek 47 prediction
id,,,,,,,,,,,,,,,,,,,,,
4,50.411000,3.600786,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,10.500000,3.600000,3.800000,2.400000,6.800000,4.071000,2.988724,3,0.0
5,51.013493,2.550675,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.300000,1.000000,NaN,NaN,NaN,NaN,NaN,3.220320,3,0.0
6,90.628000,3.125103,8,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,NaN,NaN,NaN,1.000000,3.600000,2.000000,3.715550,0,0.0
7,62.621385,2.408515,2,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,1.305517,NaN,NaN,2.332871,NaN,NaN,NaN,3.179022,10,0.0
12,26.581693,2.044746,2,NaN,NaN,0.0,0.0,0.0,1.0,NaN,...,2.005517,NaN,NaN,NaN,NaN,NaN,NaN,2.656975,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,72.156162,2.672450,1,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,NaN,4.418731,2.382085,NaN,5.048812,NaN,NaN,1.886890,0,0.0
470,70.039342,3.183606,4,NaN,1.0,0.0,0.0,0.0,0.0,0.0,...,6.690127,NaN,NaN,NaN,NaN,NaN,3.861196,2.139477,4,0.0
475,132.625196,3.684033,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,5.274923,0.328340,11.375777,4.195247,3.331484,3.961196,3.680471,3,0.0


In [5]:
optimize(df, 86, f'gameweek {gameweek+1} prediction')

Status: Optimal
Cost: 85.80000000000001
Expected points: 75.9888424873352


,element_type,web_name,now_cost,games played,gameweek 38 prediction
id,,,,,
559,1,Ramsdale,51,33.0,4.947547
121,2,Thiago Silva,56,31.0,5.068476
127,2,Rüdiger,61,33.0,5.122841
135,2,Chilwell,56,6.0,5.220604
237,2,Alexander-Arnold,84,31.0,8.264379
361,2,Dier,47,34.0,5.226481
233,3,Salah,131,34.0,6.363566
240,3,Jota,83,34.0,5.658581
359,3,Son,112,34.0,8.705607


In [6]:
optimize(df, 86, 'adjusted points per game')

Status: Optimal
Cost: 86.0
Expected points per week: 68.33514872251057


,element_type,web_name,now_cost,games played,adjusted points per game
id,,,,,
156,1,Butland,44,9.0,4.242141
135,2,Chilwell,56,6.0,6.165082
142,2,James,64,25.0,5.139968
237,2,Alexander-Arnold,84,31.0,6.203325
256,2,Cancelo,72,35.0,5.689493
360,2,Doherty,45,15.0,4.877754
233,3,Salah,131,34.0,7.269846
255,3,Sterling,105,29.0,5.691066
265,3,Foden,79,27.0,5.200430


In [7]:
optimize(df, 86, f'xPoints_next10')

Status: Optimal
Cost: 85.80000000000001
Expected points per week: 75.9888424873352


,element_type,web_name,now_cost,games played,xPoints_next10
id,,,,,
559,1,Ramsdale,51,33.0,4.947547
121,2,Thiago Silva,56,31.0,5.068476
127,2,Rüdiger,61,33.0,5.122841
135,2,Chilwell,56,6.0,5.220604
237,2,Alexander-Arnold,84,31.0,8.264379
361,2,Dier,47,34.0,5.226481
233,3,Salah,131,34.0,6.363566
240,3,Jota,83,34.0,5.658581
359,3,Son,112,34.0,8.705607


In [8]:
df = df.drop([135])
optimize(df, 86, f'gameweek {gameweek+1} prediction')

Status: Optimal
Cost: 86.0
Expected points: 75.55250835418701


,element_type,web_name,now_cost,games played,gameweek 38 prediction
id,,,,,
69,1,Sánchez,45,36.0,4.837516
121,2,Thiago Silva,56,31.0,5.068476
23,2,Gabriel,53,34.0,4.766647
237,2,Alexander-Arnold,84,31.0,8.264379
256,2,Cancelo,72,35.0,5.250495
361,2,Dier,47,34.0,5.226481
233,3,Salah,131,34.0,6.363566
240,3,Jota,83,34.0,5.658581
359,3,Son,112,34.0,8.705607


In [9]:
df = df.drop([96,240])
optimize(df, 86, f'xPoints_next10')

Status: Optimal
Cost: 85.5
Expected points per week: 75.32807779312134


,element_type,web_name,now_cost,games played,xPoints_next10
id,,,,,
69,1,Sánchez,45,36.0,4.837516
121,2,Thiago Silva,56,31.0,5.068476
127,2,Rüdiger,61,33.0,5.122841
229,2,van Dijk,68,34.0,5.556205
23,2,Gabriel,53,34.0,4.766647
237,2,Alexander-Arnold,84,31.0,8.264379
123,3,Jorginho,57,29.0,4.656547
233,3,Salah,131,34.0,6.363566
359,3,Son,112,34.0,8.705607


In [10]:
df = df.drop([23])
optimize(df, 86, f'xPoints_next10')

Status: Optimal
Cost: 86.0
Expected points per week: 75.30905437469482


,element_type,web_name,now_cost,games played,xPoints_next10
id,,,,,
559,1,Ramsdale,51,33.0,4.947547
121,2,Thiago Silva,56,31.0,5.068476
127,2,Rüdiger,61,33.0,5.122841
229,2,van Dijk,68,34.0,5.556205
237,2,Alexander-Arnold,84,31.0,8.264379
361,2,Dier,47,34.0,5.226481
123,3,Jorginho,57,29.0,4.656547
233,3,Salah,131,34.0,6.363566
359,3,Son,112,34.0,8.705607
